# Task 1: Data Exploration and Enrichment
## Ethiopia Financial Inclusion Forecasting

**Objective:** Understand the starter dataset and enrich it with additional data for forecasting.

### Tasks:
1. Understand the Schema
2. Explore the Data  
3. Enrich the Dataset
4. Follow the Schema
5. Document Your Additions

In [2]:
# Setup and imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

print("✅ Libraries imported")

✅ Libraries imported


In [3]:
# Load the two CSV files
main_df = pd.read_csv('../data/raw/ethiopia_fi_unified_data.csv')
ref_df = pd.read_csv('../data/raw/reference_codes.csv')

print("📊 Data Loaded Successfully")
print(f"Main dataset: {main_df.shape[0]} rows, {main_df.shape[1]} columns")
print(f"Reference codes: {ref_df.shape[0]} rows, {ref_df.shape[1]} columns")
print("\nMain dataset columns:", list(main_df.columns))

📊 Data Loaded Successfully
Main dataset: 43 rows, 34 columns
Reference codes: 71 rows, 4 columns

Main dataset columns: ['record_id', 'record_type', 'category', 'pillar', 'indicator', 'indicator_code', 'indicator_direction', 'value_numeric', 'value_text', 'value_type', 'unit', 'observation_date', 'period_start', 'period_end', 'fiscal_year', 'gender', 'location', 'region', 'source_name', 'source_type', 'source_url', 'confidence', 'related_indicator', 'relationship_type', 'impact_direction', 'impact_magnitude', 'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country', 'collected_by', 'collection_date', 'original_text', 'notes']


# Task 1.1  Understand the Schema

The unified schema uses `record_type` to differentiate:
- `observation`: Measured values
- `event`: Policies, product launches, milestones
- `impact_link`: Relationships between events and indicators
- `target`: Official policy goals

**Key Design Principle:** Events are NOT pre-assigned to pillars. Their effects are captured through `impact_link` records.

In [4]:
print("Exploring recoed types...")
# Record type distribution
print("📈 RECORD TYPE DISTRIBUTION")
print("=" * 50)
record_counts = main_df['record_type'].value_counts()
print(record_counts)

print("\n📋 SAMPLE OF EACH RECORD TYPE")
print("=" * 50)

for rt in main_df['record_type'].unique():
    print(f"\n--- {rt.upper()} ---")
    sample = main_df[main_df['record_type'] == rt].head(2)
    # Select relevant columns for display
    display_cols = [c for c in ['record_id', 'indicator', 'value_numeric', 
                               'observation_date', 'category', 'pillar'] 
                   if c in sample.columns]
    print(sample[display_cols].to_string(index=False))

Exploring recoed types...
📈 RECORD TYPE DISTRIBUTION
record_type
observation    30
event          10
target          3
Name: count, dtype: int64

📋 SAMPLE OF EACH RECORD TYPE

--- OBSERVATION ---
record_id              indicator  value_numeric observation_date category pillar
 REC_0001 Account Ownership Rate           22.0       2014-12-31      NaN ACCESS
 REC_0002 Account Ownership Rate           35.0       2017-12-31      NaN ACCESS

--- TARGET ---
record_id                   indicator  value_numeric observation_date category pillar
 REC_0031      Account Ownership Rate           70.0       2025-12-31      NaN ACCESS
 REC_0032 Fayda Digital ID Enrollment     90000000.0       2028-12-31      NaN ACCESS

--- EVENT ---
record_id                            indicator  value_numeric observation_date       category pillar
 EVT_0001                      Telebirr Launch            NaN       2021-05-17 product_launch    NaN
 EVT_0002 Safaricom Ethiopia Commercial Launch            NaN       20

In [5]:
print("Exploring record types...")
# Explore reference codes
print("🔤 REFERENCE CODES EXPLORATION")
print("=" * 50)
print(f"Unique fields: {ref_df['field'].nunique()}")
print("\nField distribution:")
print(ref_df['field'].value_counts())

# Show samples of important fields
print("\n📝 SAMPLES OF KEY FIELDS:")
for field in ['record_type', 'category', 'pillar', 'indicator_code']:
    if field in ref_df['field'].values:
        samples = ref_df[ref_df['field'] == field].head(3)
        print(f"\n{field}:")
        for _, row in samples.iterrows():
            print(f"  {row['code']}: {row['description']}")

Exploring record types...
🔤 REFERENCE CODES EXPLORATION
Unique fields: 13

Field distribution:
field
value_type             11
category               10
source_type             8
pillar                  7
record_type             6
confidence              4
relationship_type       4
impact_direction        4
impact_magnitude        4
evidence_basis          4
indicator_direction     3
gender                  3
location                3
Name: count, dtype: int64

📝 SAMPLES OF KEY FIELDS:

record_type:
  observation: Actual measured value from a source
  event: Policy launch market event or milestone
  impact_link: Relationship between event and indicator (links via parent_id)

category:
  product_launch: New product or service introduced
  market_entry: New competitor enters market
  market_exit: Competitor leaves market

pillar:
  ACCESS: Can people reach services? Coverage devices accounts
  USAGE: Are people actively using? Transactions active users
  QUALITY: Do services work? Succes

## Task 1.2 Explore the Data

I'll examine:
- Count records by record_type, pillar, source_type, and confidence
- Temporal range of observations
- Unique indicators and their coverage
- Cataloged events and their dates
- Existing impact links and relationships

In [7]:
print("Counting the records with values of..")
record_counts = (
    main_df.groupby(["record_type", "pillar", "source_type", "confidence"])
      .size()  # counts rows in each group
      .reset_index(name="count")  # turn into DataFrame with a 'count' column
)

# Display the result
record_counts


Counting the records with values of..


,record_type,pillar,source_type,confidence,count
0,observation,ACCESS,operator,high,2
1,observation,ACCESS,regulator,high,2
2,observation,ACCESS,research,high,2
3,observation,ACCESS,survey,high,8
4,observation,AFFORDABILITY,research,medium,1
5,observation,GENDER,regulator,high,1
6,observation,GENDER,research,high,1
7,observation,GENDER,survey,high,1
8,observation,GENDER,survey,medium,1
9,observation,USAGE,calculated,high,2


In [8]:
print("Temporal analysis...")
# Temporal analysis
print("📅 TEMPORAL ANALYSIS")
print("=" * 50)

# Check date columns
date_cols = [col for col in main_df.columns if 'date' in col.lower() or 'year' in col.lower()]
print(f"Date-related columns: {date_cols}")

if 'observation_date' in main_df.columns:
    # Parse dates
    main_df['observation_date_parsed'] = pd.to_datetime(main_df['observation_date'], errors='coerce')
    
    # Get date range
    valid_dates = main_df['observation_date_parsed'].dropna()
    if not valid_dates.empty:
        print(f"\nDate Range: {valid_dates.min()} to {valid_dates.max()}")
        
        # Count by year
        main_df['year'] = main_df['observation_date_parsed'].dt.year
        year_counts = main_df['year'].value_counts().sort_index()
        print("\nRecords by year:")
        print(year_counts)
        
        # Observations by year
        obs_by_year = main_df[main_df['record_type'] == 'observation'].groupby('year').size()
        print("\nObservations by year:")
        print(obs_by_year)

Temporal analysis...
📅 TEMPORAL ANALYSIS
Date-related columns: ['observation_date', 'fiscal_year', 'collection_date']

Date Range: 2014-12-31 00:00:00 to 2030-12-31 00:00:00

Records by year:
year
2014     1
2017     1
2021     7
2022     1
2023     2
2024    14
2025    15
2028     1
2030     1
Name: count, dtype: int64

Observations by year:
year
2014     1
2017     1
2021     5
2023     1
2024    11
2025    11
dtype: int64


In [10]:
print("Listing all unique indicators...")
# Indicators and pillars analysis
print("📊 INDICATORS AND PILLARS")
print("=" * 50)

# For observations
obs_df = main_df[main_df['record_type'] == 'observation']
print(f"Total observations: {obs_df.shape[0]}")

# Pillar distribution
if 'pillar' in obs_df.columns:
    print("\nObservations by pillar:")
    print(obs_df['pillar'].value_counts())

# Indicator coverage
if 'indicator_code' in obs_df.columns:
    print(f"\nUnique indicators: {obs_df['indicator_code'].nunique()}")
    print("\nTop 10 indicators:")
    print(obs_df['indicator_code'].value_counts().head(10))

Listing all unique indicators...
📊 INDICATORS AND PILLARS
Total observations: 30

Observations by pillar:
pillar
ACCESS           14
USAGE            11
GENDER            4
AFFORDABILITY     1
Name: count, dtype: int64

Unique indicators: 19

Top 10 indicators:
indicator_code
ACC_OWNERSHIP      6
ACC_FAYDA          3
ACC_4G_COV         2
USG_P2P_COUNT      2
GEN_GAP_ACC        2
ACC_MM_ACCOUNT     2
USG_MPESA_USERS    1
GEN_MM_SHARE       1
AFF_DATA_INCOME    1
USG_ACTIVE_RATE    1
Name: count, dtype: int64


In [11]:
print("Undestanding events that are catalogued and their data...")
# Events analysis
print("🎯 EVENTS ANALYSIS")
print("=" * 50)

events_df = main_df[main_df['record_type'] == 'event']
print(f"Total events: {events_df.shape[0]}")

if 'category' in events_df.columns:
    print("\nEvents by category:")
    print(events_df['category'].value_counts())

# Show event timeline
if 'observation_date' in events_df.columns:
    print("\nEvent timeline:")
    event_timeline = events_df[['observation_date', 'indicator', 'category']].sort_values('observation_date')
    print(event_timeline.to_string(index=False))

Undestanding events that are catalogued and their data...
🎯 EVENTS ANALYSIS
Total events: 10

Events by category:
category
product_launch    2
infrastructure    2
policy            2
market_entry      1
milestone         1
partnership       1
pricing           1
Name: count, dtype: int64

Event timeline:
observation_date                              indicator       category
      2021-05-17                        Telebirr Launch product_launch
      2021-09-01                NFIS-II Strategy Launch         policy
      2022-08-01   Safaricom Ethiopia Commercial Launch   market_entry
      2023-08-01                 M-Pesa Ethiopia Launch product_launch
      2024-01-01       Fayda Digital ID Program Rollout infrastructure
      2024-07-29        Foreign Exchange Liberalization         policy
      2024-10-01    P2P Transaction Count Surpasses ATM      milestone
      2025-10-27           M-Pesa EthSwitch Integration    partnership
      2025-12-15      Safaricom Ethiopia Price Increase

In [12]:
print("Reviewing the existing impact_links...")
# Impact links analysis
print("🔗 IMPACT LINKS ANALYSIS")
print("=" * 50)

impact_df = main_df[main_df['record_type'] == 'impact_link']
print(f"Total impact links: {impact_df.shape[0]}")

if 'pillar' in impact_df.columns:
    print("\nImpact links by pillar:")
    print(impact_df['pillar'].value_counts())

# Show relationships
if 'impact_direction' in impact_df.columns:
    print("\nImpact directions:")
    print(impact_df['impact_direction'].value_counts())

if 'impact_magnitude' in impact_df.columns:
    print("\nImpact magnitudes:")
    print(impact_df['impact_magnitude'].value_counts())

Reviewing the existing impact_links...
🔗 IMPACT LINKS ANALYSIS
Total impact links: 0

Impact links by pillar:
Series([], Name: count, dtype: int64)

Impact directions:
Series([], Name: count, dtype: int64)

Impact magnitudes:
Series([], Name: count, dtype: int64)


## Data Quality Assessment

### Issues Identified:
1. **Sparse temporal data**: Only 5 data points for ACCESS pillar (2011, 2014, 2017, 2021, 2024)
2. **Missing years**: Gaps between survey years need interpolation
3. **Mixed data types**: Some numeric values may be stored as text
4. **Event-impact relationships**: Need validation against actual outcomes

In [13]:
# Data quality checks
print("🔍 DATA QUALITY CHECKS")
print("=" * 50)

# 1. Check for missing values
print("Missing values by column:")
missing_pct = (main_df.isnull().sum() / len(main_df) * 100).round(2)
print(missing_pct[missing_pct > 0].sort_values(ascending=False))

# 2. Check data types
print("\nData types:")
print(main_df.dtypes)

# 3. Check for duplicates
print(f"\nDuplicate records: {main_df.duplicated().sum()}")

# 4. Check value consistency
if 'value_numeric' in main_df.columns:
    print("\nValue numeric summary:")
    print(main_df['value_numeric'].describe())
    
# 5. Check confidence levels
if 'confidence' in main_df.columns:
    print("\nConfidence level distribution:")
    print(main_df['confidence'].value_counts())

🔍 DATA QUALITY CHECKS
Missing values by column:
related_indicator      100.00
region                 100.00
evidence_basis         100.00
lag_months             100.00
impact_estimate        100.00
impact_magnitude       100.00
impact_direction       100.00
relationship_type      100.00
notes                  100.00
period_end              76.74
period_start            76.74
value_text              76.74
collection_date         76.74
category                76.74
source_url              27.91
pillar                  23.26
unit                    23.26
value_numeric           23.26
indicator_direction     23.26
original_text           23.26
dtype: float64

Data types:
record_id                          object
record_type                        object
category                           object
pillar                             object
indicator                          object
indicator_code                     object
indicator_direction                object
value_numeric                 

## 3. Enrich the Dataset

### Data Sources Needed:
From the Additional Data Points Guide:

#### A. Alternative Baselines:
- IMF FAS, G20 indicators, GSMA, ITU, NBE reports

#### B. Direct Correlation Indicators:
- Active accounts, agent density, POS terminals, transaction volumes

#### C. Indirect Correlation (Proxies):
- Smartphone penetration, data affordability, urbanization, digital ID

#### D. Market Nuances (Ethiopia-specific):
- P2P dominance, mobile money-only users rare, low credit penetration

In [14]:
# Create enrichment plan
enrichment_plan = {
    'additional_observations': [
        'Quarterly mobile money users (2021-2024)',
        'Monthly agent network growth',
        'Smartphone penetration (annual)',
        '4G coverage expansion',
        'Digital ID (Fayda) registration'
    ],
    'additional_events': [
        'Specific regulatory changes (2022-2024)',
        'Bank-Mobile money partnerships',
        'Pricing changes for services',
        'Infrastructure milestones',
        'Regional expansion events'
    ],
    'additional_impact_links': [
        'Validate Telebirr impact estimates',
        'Add M-Pesa market entry impacts',
        'Model infrastructure effects',
        'Include economic factors (inflation, FX)'
    ]
}

print("📝 ENRICHMENT PLAN")
print("=" * 50)
for category, items in enrichment_plan.items():
    print(f"\n{category.replace('_', ' ').title()}:")
    for item in items:
        print(f"  • {item}")

📝 ENRICHMENT PLAN

Additional Observations:
  • Quarterly mobile money users (2021-2024)
  • Monthly agent network growth
  • Smartphone penetration (annual)
  • 4G coverage expansion
  • Digital ID (Fayda) registration

Additional Events:
  • Specific regulatory changes (2022-2024)
  • Bank-Mobile money partnerships
  • Pricing changes for services
  • Infrastructure milestones
  • Regional expansion events

Additional Impact Links:
  • Validate Telebirr impact estimates
  • Add M-Pesa market entry impacts
  • Model infrastructure effects
  • Include economic factors (inflation, FX)


## 4. Document Additions

We'll create a `data_enrichment_log.md` file to track all additions.

In [15]:
# Create data enrichment log template
log_content = """# Data Enrichment Log

## New Records Added

### Observations Added:
| record_id | pillar | indicator | indicator_code | value_numeric | observation_date | source_name | source_url | confidence | collected_by | collection_date | notes |
|-----------|--------|-----------|----------------|---------------|------------------|-------------|------------|------------|--------------|-----------------|-------|

### Events Added:
| record_id | category | indicator | observation_date | source_name | source_url | confidence | collected_by | collection_date | notes |
|-----------|----------|-----------|------------------|-------------|------------|------------|--------------|-----------------|-------|

### Impact Links Added:
| record_id | parent_id | pillar | related_indicator | impact_direction | impact_magnitude | lag_months | evidence_basis | confidence | collected_by | collection_date | notes |
|-----------|-----------|--------|-------------------|------------------|------------------|------------|----------------|------------|--------------|-----------------|-------|

## Summary of Additions
- Total new observations: 
- Total new events: 
- Total new impact links: 

## Data Sources Used
1. 
2. 
3. 

## Challenges and Limitations
1. 
2. 
3. 
"""

# Save to file
with open('../data_enrichment_log.md', 'w') as f:
    f.write(log_content)

print("✅ Created data_enrichment_log.md template")

✅ Created data_enrichment_log.md template


In [16]:
# Function to save enriched dataset (to be filled with actual data)
def save_enriched_dataset(original_df, new_records, output_path):
    """
    Combine original data with new records and save.
    
    Parameters:
    original_df: Original DataFrame
    new_records: List of new records (DataFrames or dicts)
    output_path: Path to save enriched dataset
    """
    # Convert new records to DataFrame
    new_df = pd.DataFrame(new_records)
    
    # Combine with original
    enriched_df = pd.concat([original_df, new_df], ignore_index=True)
    
    # Save to CSV
    enriched_df.to_csv(output_path, index=False)
    print(f"✅ Enriched dataset saved to {output_path}")
    print(f"   Original: {len(original_df)} records")
    print(f"   Added: {len(new_df)} records")
    print(f"   Total: {len(enriched_df)} records")
    
    return enriched_df

# Example usage (commented out until we have actual data)
"""
# When you have actual new records:
new_observations = [...]  # List of observation dicts
new_events = [...]  # List of event dicts  
new_impacts = [...]  # List of impact link dicts

all_new_records = new_observations + new_events + new_impacts
enriched_df = save_enriched_dataset(main_df, all_new_records, '../data/raw/ethiopia_fi_unriched_enriched.csv')
"""
print("📁 Enrichment function ready - add actual data to use it")

📁 Enrichment function ready - add actual data to use it


In [17]:
# Final summary statistics
print("📈 TASK 1 SUMMARY")
print("=" * 50)

print(f"\nMain Dataset: {main_df.shape[0]} rows × {main_df.shape[1]} columns")
print(f"Reference Codes: {ref_df.shape[0]} rows × {ref_df.shape[1]} columns")

print("\n📊 Record Type Distribution:")
print(main_df['record_type'].value_counts())

print("\n📅 Temporal Coverage:")
if 'year' in main_df.columns:
    print(f"Years with data: {sorted(main_df['year'].dropna().unique())}")

print("\n🔍 Data Quality:")
print(f"Missing values: {main_df.isnull().sum().sum()} total")
print(f"Duplicate rows: {main_df.duplicated().sum()}")

print("\n✅ Task 1 Exploration Complete!")
print("\nNext: Add actual enrichment data to the dataset.")

📈 TASK 1 SUMMARY

Main Dataset: 43 rows × 36 columns
Reference Codes: 71 rows × 4 columns

📊 Record Type Distribution:
record_type
observation    30
event          10
target          3
Name: count, dtype: int64

📅 Temporal Coverage:
Years with data: [2014, 2017, 2021, 2022, 2023, 2024, 2025, 2028, 2030]

🔍 Data Quality:
Missing values: 614 total
Duplicate rows: 0

✅ Task 1 Exploration Complete!

Next: Add actual enrichment data to the dataset.
